# Análise de Logs do Painel de Sensores de Incêndio

In [10]:
import pandas as pd
import re
import datetime
import plotly.express as px

In [11]:
def extrair_informacoes(conteudo):
    """Extrai informações do conteúdo do log usando regex"""
    # Padrão para encontrar as linhas de dados
    padrao = r'^\s*(\d+)\s+([\w\-\d]+)\s+([*]?[\d\.]+/\d+)\s+(\d+)\s+(\d+/\s*\d+%)\s+(\d+/\s*\d+%)\s+(\w+)'
    
    dados = []
    channel = None
    data_hora = None
    
    for linha in conteudo.split('\n'):
        # Extrair informação do canal
        canal_match = re.search(r'Channel (\d+) \((M\d+)\)', linha)
        if canal_match:
            channel = canal_match.group(1)
            continue
            
        # Extrair data e hora
        data_match = re.search(r'(\d{2}:\d{2}:\d{2})\s+(\w{3})\s+(\d{2})-(\w{3})-(\d{2})', linha)
        if data_match:
            hora = data_match.group(1)
            dia_semana = data_match.group(2)
            dia = data_match.group(3)
            mes = data_match.group(4)
            ano = data_match.group(5)
            data_hora = f"{dia}/{mes}/20{ano} {hora}"
            continue
            
        # Extrair dados dos sensores
        match = re.match(padrao, linha)
        if match:
            dev_num, label, alarm_at, avg_val, current_alarm, peak_alarm, state = match.groups()
            
            dados.append({
                'Channel': channel,
                'DateTime': data_hora,
                'DeviceNumber': dev_num.strip(),
                'Label': label.strip(),
                'AlarmAt': alarm_at.strip(),
                'AverageValue': avg_val.strip(),
                'CurrentAlarm': current_alarm.strip(),
                'PeakAlarm': peak_alarm.strip(),
                'State': state.strip()
            })
    
    return pd.DataFrame(dados)

In [12]:
def processar_valores_alarme(df):
    """Processa as colunas de alarme separando valores e percentuais"""
    
    # Função auxiliar para extrair valor numérico do percentual
    def extrair_percentual(valor):
        return int(re.search(r'(\d+)%', valor).group(1))
    
    def extrair_valor(valor):
        return int(valor.split('/')[0].strip())
    
    # Processar AlarmAt - remover o prefixo e manter apenas o valor após a /
    df['AlarmAt'] = df['AlarmAt'].apply(lambda x: x.split('/')[-1])
    
    # Processar CurrentAlarm
    df['CurrentAlarmValue'] = df['CurrentAlarm'].apply(extrair_valor)
    df['CurrentAlarmPercent'] = df['CurrentAlarm'].apply(extrair_percentual)
    
    # Processar PeakAlarm
    df['PeakAlarmValue'] = df['PeakAlarm'].apply(extrair_valor)
    df['PeakAlarmPercent'] = df['PeakAlarm'].apply(extrair_percentual)
    
    # Remover as colunas originais se desejar
    df = df.drop(['CurrentAlarm', 'PeakAlarm'], axis=1)
    
    return df

In [13]:
def ler_arquivo_log(caminho):
    """Lê o arquivo de log e retorna seu conteúdo"""
    with open(caminho, 'r', encoding='utf-8') as file:
        return file.read()

In [14]:
# Ler e processar o arquivo
caminho_arquivo = 'data/TrueAlarmService.txt'
conteudo = ler_arquivo_log(caminho_arquivo)
df = extrair_informacoes(conteudo)

# Aplicar o processamento adicional
df = processar_valores_alarme(df)

# Verificar os valores únicos da coluna AlarmAt após o processamento
print("Valores únicos em AlarmAt:")
print(df['AlarmAt'].value_counts().head())

# Exibir as primeiras linhas do DataFrame
print("\nPrimeiras linhas do DataFrame:")
df.head()

Valores únicos em AlarmAt:
136    15
143    14
142    13
137    11
145    10
Name: AlarmAt, dtype: int64

Primeiras linhas do DataFrame:


,Channel,DateTime,DeviceNumber,Label,AlarmAt,AverageValue,State,CurrentAlarmValue,CurrentAlarmPercent,PeakAlarmValue,PeakAlarmPercent
0,1,13/FEB/2025 10:11:47,18,N1-L01-018-DF-FRALDARIO-TE-PE,131,63,NOR,65,0,202,100
1,1,13/FEB/2025 10:11:49,56,N1-L01-056-DF-CIRCULACAO-TE-PE,146,78,NOR,79,0,83,4
2,1,13/FEB/2025 10:11:49,57,N1-L01-057-DF-CIRCULACAO-TE-PE,130,62,NOR,62,0,82,28
3,1,13/FEB/2025 10:11:49,58,N1-L01-058-DF-CIRCULACAO-TE-PE,122,54,NOR,54,0,255,100
4,1,13/FEB/2025 10:11:49,59,N1-L01-059-DF-CIRCULACAO-TE-PE,122,54,NOR,53,0,148,100


In [15]:
# Verificar estatísticas das novas colunas
print("\nEstatísticas dos valores de alarme:")
print("\nCurrent Alarm Values:")
print(df['CurrentAlarmValue'].describe())

print("\nCurrent Alarm Percentages:")
print(df['CurrentAlarmPercent'].describe())

print("\nPeak Alarm Values:")
print(df['PeakAlarmValue'].describe())

print("\nPeak Alarm Percentages:")
print(df['PeakAlarmPercent'].describe())


Estatísticas dos valores de alarme:

Current Alarm Values:
count    220.000000
mean      71.613636
std       11.082126
min       52.000000
25%       65.750000
50%       71.000000
75%       77.000000
max      182.000000
Name: CurrentAlarmValue, dtype: float64

Current Alarm Percentages:
count    220.000000
mean       0.454545
std        6.741999
min        0.000000
25%        0.000000
50%        0.000000
75%        0.000000
max      100.000000
Name: CurrentAlarmPercent, dtype: float64

Peak Alarm Values:
count    220.000000
mean      86.922727
std       66.451194
min        0.000000
25%       65.000000
50%       74.000000
75%       86.000000
max      255.000000
Name: PeakAlarmValue, dtype: float64

Peak Alarm Percentages:
count    220.000000
mean      18.472727
std       34.364607
min        0.000000
25%        0.000000
50%        0.000000
75%       20.000000
max      100.000000
Name: PeakAlarmPercent, dtype: float64


In [16]:
df.AlarmAt.value_counts()

136    15
143    14
142    13
137    11
145    10
140    10
146    10
138     9
131     8
139     8
148     8
132     8
130     7
135     7
141     7
134     6
150     6
144     6
133     5
129     5
127     5
149     5
128     5
126     4
122     4
151     4
152     3
147     3
123     3
155     2
153     2
159     1
165     1
120     1
118     1
154     1
156     1
157     1
Name: AlarmAt, dtype: int64

In [20]:
# Ler e processar o arquivo
caminho_arquivo = 'data/TrueAlarmService.txt'
conteudo = ler_arquivo_log(caminho_arquivo)
df = extrair_informacoes(conteudo)
df = processar_valores_alarme(df)

# Extrair localização do Label
df['Location'] = df['Label'].apply(lambda x: x.split('-')[-2] if len(x.split('-')) > 2 else '')

In [21]:
# Estatísticas por Canal
print("\nDistribuição de sensores por canal:")
print(df['Channel'].value_counts())

# Estatísticas de alarmes
print("\nEstatísticas de valores de alarme:")
print(df[['AlarmAt', 'CurrentAlarmValue', 'PeakAlarmValue']].describe())

# Contagem de estados
print("\nDistribuição de estados:")
print(df['State'].value_counts())


Distribuição de sensores por canal:
7     78
2     51
1     42
3     28
5      8
6      5
9      3
4      2
10     2
8      1
Name: Channel, dtype: int64

Estatísticas de valores de alarme:
       CurrentAlarmValue  PeakAlarmValue
count         220.000000      220.000000
mean           71.613636       86.922727
std            11.082126       66.451194
min            52.000000        0.000000
25%            65.750000       65.000000
50%            71.000000       74.000000
75%            77.000000       86.000000
max           182.000000      255.000000

Distribuição de estados:
NOR    220
Name: State, dtype: int64


In [26]:
# Identificar sensores com altos valores de pico
sensores_criticos = df[df['CurrentAlarmValue'] > 80].sort_values('CurrentAlarmValue', ascending=False)

print("Sensores com valores de pico críticos (>80%):")
sensores_criticos[['DeviceNumber', 'Label', 'CurrentAlarmValue', 'CurrentAlarmPercent', 'Location']].head(50)

Sensores com valores de pico críticos (>80%):


,DeviceNumber,Label,CurrentAlarmValue,CurrentAlarmPercent,Location
81,153,N1-L02-153-DF-WC-TE-PE,182,100,TE
110,48,N1-L03-048-DF-CAG-TE-PE,98,0,TE
59,66,N1-L02-066-DF-WC-TE-PE,92,0,TE
77,147,N1-L02-147-DF-RECEPCAO-TE-PE,88,0,TE
194,180,N1-L07-180-DF-COPA-2P-PE,87,0,2P
159,54,N1-L07-054-DF-CIRCULACAO-2P-PE,87,0,2P
153,45,N1-L07-045-DF-X-2P-PE,87,0,2P
155,47,N1-L07-047-DF-CREDENCIAMENTO-2P-PE,86,0,2P
37,236,N1-L01-236-DF-BANHEIRO-TE-PE,85,0,TE
148,33,N1-L07-033-DF-CIRCULACAO-2P-PE,85,0,2P
